# EAGLE-2 SE OmniDraft Setup and Execution

This notebook orchestrates the complete setup and execution process for the EAGLE-2 SE OmniDraft project.
The logic has been refactored into separate modules for better maintainability.

## Architecture
- `config.py`: Configuration settings
- `setup_utils.py`: Reusable setup utilities
- `requirements.txt`: Python dependencies
- This notebook: Orchestration and documentation

## Prerequisites
- Python 3.8+
- CUDA-compatible GPU (recommended)
- HuggingFace account for model access

## Step 1: Import Dependencies and Configuration

In [ ]:
import os
import sys
from pathlib import Path

# Import our custom utilities
from setup_utils import ProjectSetup, ModelManager, ScriptExecutor
import config

print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")
print(f"Repository URL: {config.REPO_URL}")

## Step 2: Environment Setup and Repository Management

In [ ]:
# Initialize setup manager
setup = ProjectSetup(config.PROJECT_DIR)

# Check environment
env_info = setup.check_environment()
print("Environment Information:")
for key, value in env_info.items():
    print(f"  {key}: {value}")

In [ ]:
# Clone repository if needed
clone_success = setup.clone_repository(config.REPO_URL)

if clone_success:
    # Change to project directory
    if os.path.exists(config.PROJECT_DIR):
        os.chdir(config.PROJECT_DIR)
        print(f"Changed to directory: {os.getcwd()}")
else:
    print("❌ Failed to clone repository. Please check the URL and try again.")

## Step 3: Install Dependencies

In [ ]:
# Install dependencies from requirements.txt
install_success = setup.install_dependencies()

if install_success:
    # Verify imports
    import_success = setup.verify_imports()
    if not import_success:
        print("❌ Some imports failed. Please check the installation.")
else:
    print("❌ Failed to install dependencies.")

## Step 4: Model Loading and Verification

In [ ]:
# Initialize model manager
model_manager = ModelManager()
print(f"Using device: {model_manager.device}")
print(f"Tiny model: {model_manager.tiny_id}")
print(f"Large model: {model_manager.large_id}")

In [ ]:
# Load TinyLlama model
tiny_tokenizer, tiny_model = model_manager.load_tiny_model()

if tiny_tokenizer and tiny_model:
    model_manager.test_tokenization(tiny_tokenizer, "TinyLlama")
else:
    print("❌ Failed to load TinyLlama model")

In [ ]:
# Load Meta-Llama model (may require authentication)
large_tokenizer, large_model = model_manager.load_large_model()

if large_tokenizer and large_model:
    model_manager.test_tokenization(large_tokenizer, "Meta-Llama")
else:
    print("❌ Failed to load Meta-Llama model")
    print("This is expected if you haven't set up HuggingFace authentication")

## Step 5: Script Execution

In [ ]:
# Initialize script executor
executor = ScriptExecutor(config.SCRIPT_NAME)

# Check if script exists
script_exists = executor.check_script_exists()

In [ ]:
# Execute script as subprocess
if script_exists:
    success, stdout, stderr = executor.execute_script(timeout=config.SCRIPT_TIMEOUT)
    
    print("STDOUT:")
    print(stdout)
    
    if stderr:
        print("\nSTDERR:")
        print(stderr)
    
    print("=" * 50)
    if success:
        print("✅ Script executed successfully")
    else:
        print("❌ Script execution failed")
else:
    print("❌ Cannot execute: script not found")

## Step 6: Interactive Execution (Alternative)

In [ ]:
# Import script as module for interactive use
if script_exists:
    hetero_module = executor.import_script_module()
    
    if hetero_module:
        # Run interactive test
        result = executor.run_interactive_test(
            hetero_module, 
            prompt=config.TEST_PROMPT
        )
        
        if result:
            print(f"\nFinal result: {result}")
else:
    print("❌ Cannot run interactive execution: script not found")

## Step 7: Custom Testing

In [ ]:
# Custom test with different parameters
if 'hetero_module' in locals() and hetero_module:
    custom_prompts = [
        "Once upon a time",
        "The key to machine learning is",
        "In the year 2030"
    ]
    
    print("Running custom tests with different prompts:")
    print("=" * 50)
    
    for i, prompt in enumerate(custom_prompts, 1):
        print(f"\nTest {i}: '{prompt}'")
        try:
            result = hetero_module.heterogeneous_spec_decode(
                prompt=prompt,
                max_new_tokens=config.DEFAULT_MAX_NEW_TOKENS,
                K=config.DEFAULT_K,
                alpha=config.DEFAULT_ALPHA
            )
            print(f"Result: {result}")
        except Exception as e:
            print(f"❌ Error: {e}")
else:
    print("❌ Module not available for custom testing")

## Summary and Next Steps

### What This Notebook Accomplished:
1. ✅ **Modular Architecture**: Extracted logic into reusable modules
2. ✅ **Environment Setup**: Automated dependency installation and verification
3. ✅ **Model Management**: Streamlined model loading with error handling
4. ✅ **Script Execution**: Both subprocess and interactive execution modes
5. ✅ **Configuration Management**: Centralized settings in `config.py`

### Project Structure:
```
EAGLE-2_SE_OmniDraft/
├── config.py                    # Configuration settings
├── setup_utils.py              # Reusable utilities
├── requirements.txt            # Python dependencies
├── heterogeneous_spd.py        # Main script
├── setup_and_run_refactored.ipynb  # This notebook
└── README.md                   # Documentation
```

### Next Steps:
- **Performance Monitoring**: Add GPU memory usage tracking
- **Logging**: Implement structured logging for better debugging
- **Testing**: Add unit tests for the utility modules
- **Documentation**: Expand README with detailed usage instructions
- **Configuration**: Add support for different model configurations

### Troubleshooting:
- **Model Loading Issues**: Check HuggingFace authentication and model availability
- **CUDA Errors**: Verify GPU drivers and PyTorch CUDA compatibility
- **Memory Issues**: Consider using model quantization or CPU execution
- **Import Errors**: Ensure all dependencies are properly installed